In [1]:
import pandas as pd
import sys
mylib = '/home/fra/Project/pyProj/mlib'

In [2]:
sys.path.append(mylib)

In [3]:
data_folder = "/home/fra/DataMart/datacentre/olddata/"
train_filename = 'ccbt_train.csv'

In [4]:
df = pd.read_csv(data_folder + train_filename, skipinitialspace=True)
df.head()

,target_f,GCIS_KEY,DATA_DT,CUNCZZ_AGE_YEARS,CUBCZZ_PERSONAL_CUST_IND,CUBCZZ_ALLOW_ADVERT_MAIL,CUBCZZ_ALLOW_PHONE_CONTACT,CUBCZZ_REL_MGT_IND,CUNCZZ_CUST_TENURE,CUCCZZ_CUST_GENDER,...,AFBCZZ_LOAN_OFFSET_IND,AFBCZZ_TD_IND,AFNCZZ_TD_BAL_AMT,AFBCZZ_TD_MATURE_IN_1M,AFBCZZ_TD_MATURE_IN_3M,AFBCZZ_TD_MATURE_IN_6M,AFBCZZ_FR_LOAN_MATURE_IN_3M,AFBCZZ_FR_LOAN_MATURE_IN_6M,AFBCZZ_FR_LOAN_MATURE_IN_12M,PROCESS_DTTM
0,0,55349465,31/10/2017,0,0,0,0,1,1.0,NaN,...,0,0,0.0,0,0,0,0,0,0,27:01.4
1,1,55510132,30/11/2017,33,1,0,0,0,1.0,F,...,1,0,0.0,0,0,0,0,0,0,37:10.8
2,0,50260882,30/11/2017,58,1,0,0,0,18.0,F,...,0,0,0.0,0,0,0,0,0,0,37:10.8
3,0,52934020,31/07/2017,46,1,1,1,0,9.0,F,...,0,0,0.0,0,0,0,0,0,0,07:09.2
4,0,54700818,31/05/2018,0,0,0,0,1,4.0,NaN,...,0,0,0.0,0,0,0,0,0,1,07:24.8


In [5]:
import utility as util
import explore as ex
import pipeline as pl
import transform as trf

In [6]:
from imp import reload

In [7]:
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
ntransformer = StandardScaler
ctransformer = OrdinalEncoder

### Pipeline

In [8]:
exclusions=['DATA_DT', 'PROCESS_DTTM', 'GCIS_KEY']

In [9]:
target_col = 'TARGET_F'

In [10]:
(X, y, features, py, px) = pl.standard_feature_preprocess(df, target_col, ntransformer, ctransformer, exclusions=['DATA_DT', 'PROCESS_DTTM', 'GCIS_KEY'])

In [11]:
X.shape, y.shape, features.shape

((40675, 361), (40675,), (361,))

### Modelling

In [12]:
# split train / test
from sklearn.model_selection import train_test_split
features = df.columns
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [13]:
import xgboost as xgb
model = xgb.XGBClassifier().fit(X_train, y_train)

In [14]:
def print_model_performance(model, X_test, y_test):
    predictions = model.predict(X_test)
    pred_prob = model.predict_proba(X_test)

    print("Accuracy: %f" % accuracy_score(y_test, predictions))
    print("Confustion Matrix: ")
    print(confusion_matrix(y_true=y_test, y_pred=predictions))
    print("AUC Score: %f" % roc_auc_score(y_test, pred_prob[:,1]))
    print("GINI Coefficient: %f" % (roc_auc_score(y_test, pred_prob[:,1])*2-1))


In [15]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import confusion_matrix

In [16]:
print_model_performance(model, X_test, y_test)

Accuracy: 0.658854
Confustion Matrix: 
[[4317 2069]
 [2094 3723]]
AUC Score: 0.725428
GINI Coefficient: 0.450857


### Dummy Transform

In [19]:
(dX, dy, dfeatures, dpy, dpx) = pl.dummy_feature_preprocess(df, target_col,exclusions=['DATA_DT', 'PROCESS_DTTM', 'GCIS_KEY'])

In [20]:
# split train / test
from sklearn.model_selection import train_test_split
features = df.columns
X_train, X_test, y_train, y_test = train_test_split(dX, dy, test_size=0.3)

In [21]:
dummy_model = xgb.XGBClassifier().fit(X_train, y_train)

In [23]:
print_model_performance(dummy_model, X_test, y_test)

Accuracy: 0.650824
Confustion Matrix: 
[[4242 2131]
 [2130 3700]]
AUC Score: 0.714111
GINI Coefficient: 0.428222


### Guided pipeline

In [30]:
from category_encoders.woe import WOEEncoder

In [32]:
transformer = WOEEncoder
(dX, dy, dfeatures, dpy, dpx) = pl.target_guided_feature_preprocess(df, target_col, transformer, exclusions=['DATA_DT', 'PROCESS_DTTM', 'GCIS_KEY'])

In [33]:
# split train / test
from sklearn.model_selection import train_test_split
features = df.columns
X_train, X_test, y_train, y_test = train_test_split(dX, dy, test_size=0.3)

In [34]:
supervised_model = xgb.XGBClassifier().fit(X_train, y_train)

In [35]:
print_model_performance(supervised_model, X_test, y_test)

Accuracy: 0.662296
Confustion Matrix: 
[[4254 2069]
 [2052 3828]]
AUC Score: 0.732340
GINI Coefficient: 0.464680


### Out of sample - WOE

In [36]:
(traindf, testdf) = train_test_split(df, test_size=0.3)

In [37]:
traindf.shape, testdf.shape

((28472, 365), (12203, 365))

In [38]:
transformer = WOEEncoder
(dX, dy, dfeatures, dpy, dpx) = pl.target_guided_feature_preprocess(traindf, target_col, transformer, exclusions=['DATA_DT', 'PROCESS_DTTM', 'GCIS_KEY'])

In [39]:
# modelling
xgbmodel = xgb.XGBClassifier().fit(dX, dy)

In [40]:
ytest = dpy.transform(testdf)

In [41]:
Xtest = dpx.transform(testdf)

In [42]:
print_model_performance(xgbmodel, Xtest.to_numpy(), ytest)

Accuracy: 0.642055
Confustion Matrix: 
[[4193 2108]
 [2260 3642]]
AUC Score: 0.706333
GINI Coefficient: 0.412667


### Out of sample - Dummy

In [112]:
reload(trf)
reload(pl)

<module 'pipeline' from '/home/fra/Project/pyProj/mlib/pipeline.py'>

In [113]:
(dX, dy, dfeatures, dpy, dpx) = pl.dummy_feature_preprocess(traindf, target_col,exclusions=['DATA_DT', 'PROCESS_DTTM', 'GCIS_KEY'])

In [114]:
dX.shape, dy.shape, dfeatures.shape

((28472, 2318), (28472,), (2318,))

In [115]:
# modelling
xgbmodel = xgb.XGBClassifier().fit(dX, dy)

In [116]:
ytest = dpy.transform(testdf)

In [117]:
Xtest = dpx.transform(testdf)

In [118]:
emptydf = pd.DataFrame(columns=dfeatures)

In [119]:
tdf = emptydf.append(Xtest)[dfeatures]

In [120]:
set(tdf).difference(dfeatures)

set()

In [121]:
set(dfeatures).difference(tdf)

set()

In [122]:
print_model_performance(xgbmodel, tdf.to_numpy(), ytest)

Accuracy: 0.574777
Confustion Matrix: 
[[1933 4368]
 [ 821 5081]]
AUC Score: 0.655528
GINI Coefficient: 0.311056


### Out of sample: Standard

In [63]:
ntransformer = StandardScaler
ctransformer = OrdinalEncoder
(X, y, features, py, px) = pl.standard_feature_preprocess(traindf, target_col, ntransformer, ctransformer, exclusions=['DATA_DT', 'PROCESS_DTTM', 'GCIS_KEY'])

In [64]:
X.shape, y.shape, features.shape

((28472, 361), (28472,), (361,))

In [65]:
# modelling
xgbmodel = xgb.XGBClassifier().fit(X, y)

In [66]:
ytest = py.transform(testdf)

In [67]:
Xtest = px.transform(testdf)

In [68]:
print_model_performance(xgbmodel, Xtest.to_numpy(), ytest)

Accuracy: 0.574613
Confustion Matrix: 
[[1997 4304]
 [ 887 5015]]
AUC Score: 0.649384
GINI Coefficient: 0.298768
